# Lab 10 Austin Nguyen

Editing lab10_base.ipynb (given by Dr. Fang) to increase the Test Accuracy.

December 6th, 2024

In [126]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [127]:
# Set random seed for reproducibility (your choice)
gen = torch.manual_seed(42)

In [128]:
# Load MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
mnist_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Splitting dataset into train and validation sets
train_dataset, val_dataset = random_split(mnist_dataset, [55000, 5000])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [129]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x
    


In [130]:
# Initialize the model, loss function, and optimizer
model = Classifier()
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)    # Label smoothing regularization
optimizer = optim.Adam(model.parameters(), lr=0.0005)   # Changed Learning Rate to 0.0005

# Training loop
num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Calculate validation loss
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in val_loader:
            output = model(data)
            val_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {running_loss / len(train_loader):.4f}, Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")


Epoch [1/15], Training Loss: 0.8384, Validation Loss: 0.7166, Validation Accuracy: 93.50%
Epoch [2/15], Training Loss: 0.6829, Validation Loss: 0.6651, Validation Accuracy: 95.04%
Epoch [3/15], Training Loss: 0.6445, Validation Loss: 0.6542, Validation Accuracy: 95.90%
Epoch [4/15], Training Loss: 0.6270, Validation Loss: 0.6341, Validation Accuracy: 96.66%
Epoch [5/15], Training Loss: 0.6152, Validation Loss: 0.6253, Validation Accuracy: 96.90%
Epoch [6/15], Training Loss: 0.6074, Validation Loss: 0.6167, Validation Accuracy: 97.08%
Epoch [7/15], Training Loss: 0.6009, Validation Loss: 0.6089, Validation Accuracy: 97.30%
Epoch [8/15], Training Loss: 0.5968, Validation Loss: 0.6184, Validation Accuracy: 97.18%
Epoch [9/15], Training Loss: 0.5919, Validation Loss: 0.6091, Validation Accuracy: 97.26%
Epoch [10/15], Training Loss: 0.5880, Validation Loss: 0.6079, Validation Accuracy: 97.52%
Epoch [11/15], Training Loss: 0.5865, Validation Loss: 0.6087, Validation Accuracy: 97.54%
Epoch [1

In [131]:
model.eval()

# Evaluate the model on the test set
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 97.84%
